In [7]:
# this code reference to https://www.kaggle.com/code/lunamcbride24/covid19-tweet-truth-analysis
import numpy as np
import pandas as pd
import nltk 
from nltk.corpus import stopwords
import re 
import html  
import string
import tensorflow as tf  
from tensorflow.keras.preprocessing.text import Tokenizer  
from tensorflow.keras.preprocessing.sequence import pad_sequences  
import tensorflow.keras.layers as L  
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy  
from sklearn.model_selection import train_test_split
import os

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# BEGIN AT HERE

In [9]:
!pip install pandas jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import tensorflow as tf
import keras
from keras.layers import *
from keras import Model
from keras.utils.np_utils import to_categorical
from keras import Sequential
import pandas as pd
import numpy as np
import jieba

In [11]:
def TextCNN():#reference as https://arxiv.org/pdf/1408.5882.pdf
    model = Sequential()
    model.add(Embedding(50001, 512, input_length=256)) #使用Embeeding层将每个词编码转换为词向量
    model.add(Conv1D(512, 5, padding='same'))
    #https://www.programcreek.com/python/example/89676/keras.layers.Conv1D
    model.add(MaxPooling1D(3, 3, padding='same'))
    #https://keras.io/api/layers/pooling_layers/max_pooling1d/
    model.add(Conv1D(128, 5, padding='same'))
    model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Conv1D(64, 3, padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(BatchNormalization())  # (批)规范化层
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model


def TextCNN2():
    main_input = Input(shape=(256,), dtype='float64')
    embedder = Embedding(50000 + 1, 300, input_length=256, trainable=False)
    embed = embedder(main_input)
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=48)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=47)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=46)(cnn3)
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.1)(flat)
    main_output = Dense(1, activation='relu')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [12]:
df_train = pd.read_csv("train_dataset_mixed.csv")
df_test = pd.read_csv("test_dataset_mixed.csv")
col = df_train.columns

word2bool = lambda x: 1 if x == 'real' else 0
word2str = lambda x:str(x)

df_train[col[0]] = df_train[col[0]].apply(word2str)
df_train[col[1]] = df_train[col[1]].apply(word2bool)

df_test[col[0]] = df_test[col[0]].apply(word2str)
df_test[col[1]] = df_test[col[1]].apply(word2bool)

X_train = df_train[col[0]].values.tolist()
Y_train = df_train[col[1]].values.tolist()

X_test = df_test[col[0]].values.tolist()
Y_test = df_test[col[1]].values.tolist()

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

print(X_test[:5])
print(Y_test[:5])

['RT @CDCemergency: Changes in food availability in some communities due to the #COVID19 pandemic may limit availability of fresh #food. Findâ\x80?real'
 'Yes, Doctors Confirm, You Can Catch Coronavirus From Farts! https://t.co/AiqwtuEmVY #coronavirus #obesity #farting #tacobell #mississippi'
 'nan'
 'Prescription issued by Dr Raj Kamal Agarwal (Sir Ganga Ram hospital) for general public in case of any emergency, kindly save this prescription for any kind of emergency.'
 'The science is clear." Meharryâ\x80\x99s Dr. James Hildreth says despite CDC backtrack, COVID-19 is airborne']
[0 0 0 0 0]


In [13]:
tokenizer=Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

In [14]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train,256)
X_test = pad_sequences(X_test,256)

In [15]:
model = TextCNN2()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 256, 300)     15000300    ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 256, 256)     230656      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 256, 256)     307456      ['embedding[0][0]']              
                                                                                              

In [16]:
model.fit(X_train,Y_train,batch_size=256,epochs=10,validation_data=(X_test,Y_test))

Epoch 1/10
85/85 [==============================] - 394s 5s/step - loss: 0.7498 - accuracy: 0.5954 - val_loss: 0.5645 - val_accuracy: 0.7285
Epoch 2/10
85/85 [==============================] - 382s 4s/step - loss: 0.6145 - accuracy: 0.6475 - val_loss: 0.5031 - val_accuracy: 0.7616
Epoch 3/10
85/85 [==============================] - 378s 4s/step - loss: 0.5749 - accuracy: 0.6743 - val_loss: 0.5077 - val_accuracy: 0.7794
Epoch 4/10
85/85 [==============================] - 378s 4s/step - loss: 0.5424 - accuracy: 0.7016 - val_loss: 0.4746 - val_accuracy: 0.7925
Epoch 5/10
85/85 [==============================] - 378s 4s/step - loss: 0.5092 - accuracy: 0.7212 - val_loss: 0.4641 - val_accuracy: 0.8030
Epoch 6/10
85/85 [==============================] - 382s 4s/step - loss: 0.4830 - accuracy: 0.7338 - val_loss: 0.4631 - val_accuracy: 0.8062
Epoch 7/10
85/85 [==============================] - 380s 4s/step - loss: 0.4660 - accuracy: 0.7393 - val_loss: 0.5884 - val_accuracy: 0.8041
Epoch 8/10
85

In [17]:
model.save("TextCNN.h5")